In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras
from keras import layers
import numpy as np
import gym

In [2]:
from vae_recurrent import VAE, create_decoder, create_encoder
from transition_gru import TransitionGRU
from recurrent_agent import DAIFAgentRecurrent

In [3]:
from util import random_observation_sequence, transform_observations

In [4]:
enc = create_encoder(2, 2, [20])
dec = create_decoder(2, 2, [20])
# tran = TransitionGRU(2, 1)

env = gym.make('MountainCarContinuous-v0')

Metal device set to: Apple M1 Pro


2022-08-05 19:47:59.275054: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-05 19:47:59.275177: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
num_seqs = 1200
seq_length = 12
ob_seqs = []
next_obs = []

for i in range(num_seqs):
    o, a, r = random_observation_sequence(env, seq_length)

    train = np.concatenate([o[:-1], a], axis=1)
    test = o[-1]

    ob_seqs.append(train)
    next_obs.append(test)

ob_seqs = np.array(ob_seqs)
next_obs = np.array(next_obs)
ob_seqs.shape

ob_seqs_stddev = np.ones_like(ob_seqs)
next_obs_stddev = np.ones_like(next_obs)

next_obs

array([[-0.50840491,  0.00305181],
       [-0.50073713, -0.00104602],
       [-0.50111079, -0.0097131 ],
       ...,
       [-0.46355808, -0.0072994 ],
       [-0.52336305,  0.00439201],
       [-0.52230954,  0.00199463]])

In [6]:
m = TransitionGRU(2, 1, 10, 30, 2)

m.compile(optimizer="Adam")
# m.build((None, None, 3))
# m.summary()
init_state = np.zeros((1200, 30))

In [25]:
m.fit((ob_seqs, init_state), (next_obs, next_obs_stddev), batch_size=24, epochs=20)

Epoch 1/20


2022-08-05 12:05:31.698212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-05 12:05:31.800168: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 8/50 [===>..........................] - ETA: 0s - kl_loss: 0.0897 

2022-08-05 12:05:31.908223: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 1s 8ms/step - kl_loss: 0.0428
Epoch 2/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 7.8186e-04
Epoch 3/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 1.3788e-04
Epoch 4/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 7.5271e-05
Epoch 5/20
50/50 [==============================] - 0s 7ms/step - kl_loss: 6.4968e-05
Epoch 6/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 5.8177e-05
Epoch 7/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 5.2898e-05
Epoch 8/20
50/50 [==============================] - 0s 7ms/step - kl_loss: 4.8413e-05
Epoch 9/20
50/50 [==============================] - 0s 7ms/step - kl_loss: 4.3894e-05
Epoch 10/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 3.9381e-05
Epoch 11/20
50/50 [==============================] - 0s 7ms/step - kl_loss: 3.6281e-05
Epoch 12/20
50/50 [==============================] - 0s 7ms/step - 

In [32]:
res = m((ob_seqs[0:1], None))
res[3]

<tf.Tensor: shape=(1, 12, 30), dtype=float32, numpy=
array([[[-0.04653465, -0.04555459, -0.1264562 ,  0.09157347,
          0.08645334, -0.01161919, -0.03124051,  0.00628531,
         -0.01876677, -0.08953948,  0.01370354, -0.0010269 ,
          0.13280791, -0.0102782 , -0.05815428,  0.07444919,
          0.04029816, -0.07622987,  0.06161459, -0.00730191,
          0.06826576,  0.10570525,  0.03631176,  0.01300733,
         -0.07617977, -0.08929664,  0.07647785, -0.07727528,
         -0.01774083,  0.07626397],
        [-0.02689319, -0.11230602, -0.11106204,  0.05868537,
          0.10455961,  0.02713206,  0.13228741, -0.02897012,
         -0.05533361, -0.07269165,  0.02899782,  0.06728062,
          0.09858271, -0.04127914, -0.07770452,  0.12069084,
          0.02927887, -0.09142958,  0.06304795, -0.02826957,
          0.0659746 ,  0.103673  ,  0.11727359,  0.07382652,
         -0.01626572, -0.04237746,  0.0559689 , -0.06688105,
          0.05376701,  0.09337547],
        [-0.01033003,

In [35]:
tf.squeeze(res[3])

<tf.Tensor: shape=(12, 30), dtype=float32, numpy=
array([[-0.04653465, -0.04555459, -0.1264562 ,  0.09157347,  0.08645334,
        -0.01161919, -0.03124051,  0.00628531, -0.01876677, -0.08953948,
         0.01370354, -0.0010269 ,  0.13280791, -0.0102782 , -0.05815428,
         0.07444919,  0.04029816, -0.07622987,  0.06161459, -0.00730191,
         0.06826576,  0.10570525,  0.03631176,  0.01300733, -0.07617977,
        -0.08929664,  0.07647785, -0.07727528, -0.01774083,  0.07626397],
       [-0.02689319, -0.11230602, -0.11106204,  0.05868537,  0.10455961,
         0.02713206,  0.13228741, -0.02897012, -0.05533361, -0.07269165,
         0.02899782,  0.06728062,  0.09858271, -0.04127914, -0.07770452,
         0.12069084,  0.02927887, -0.09142958,  0.06304795, -0.02826957,
         0.0659746 ,  0.103673  ,  0.11727359,  0.07382652, -0.01626572,
        -0.04237746,  0.0559689 , -0.06688105,  0.05376701,  0.09337547],
       [-0.01033003, -0.17333214, -0.07171062,  0.01438317,  0.10286073,

In [9]:
next_obs[0:10]

array([[-0.47832373, -0.00274939],
       [-0.49251005, -0.005223  ],
       [-0.57791984, -0.00141938],
       [-0.47088268, -0.00629681],
       [-0.55019915, -0.00119392],
       [-0.45699415, -0.00656845],
       [-0.54774845,  0.00525958],
       [-0.4956426 ,  0.00690681],
       [-0.52634752,  0.00604456],
       [-0.47780034, -0.00620174]])

## Manually define a good policy

In [8]:
env = gym.make('MountainCarContinuous-v0')

env.reset()

while True:

    action = float(input())
    action = np.array([action])

    print(env.step(action))

(array([-0.46002594, -0.00198855], dtype=float32), -0.1, False, {})
(array([-0.4639884 , -0.00396246], dtype=float32), -0.1, False, {})
(array([-0.46989554, -0.00590716], dtype=float32), -0.1, False, {})
(array([-0.47770372, -0.00780819], dtype=float32), -0.1, False, {})
(array([-0.48735502, -0.00965132], dtype=float32), -0.1, False, {})
(array([-0.49877763, -0.01142261], dtype=float32), -0.1, False, {})
(array([-0.51188624, -0.0131086 ], dtype=float32), -0.1, False, {})
(array([-0.52658266, -0.01469642], dtype=float32), -0.1, False, {})
(array([-0.5427567 , -0.01617404], dtype=float32), -0.1, False, {})
(array([-0.5602871 , -0.01753044], dtype=float32), -0.1, False, {})
(array([-0.579043  , -0.01875583], dtype=float32), -0.1, False, {})
(array([-0.5988849 , -0.01984191], dtype=float32), -0.1, False, {})
(array([-0.61966693, -0.02078206], dtype=float32), -0.1, False, {})
(array([-0.6412384 , -0.02157148], dtype=float32), -0.1, False, {})
(array([-0.6634458 , -0.02220738], dtype=float32

ValueError: could not convert string to float: ''